In [ ]:
###################################################

In [329]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm
import statsmodels.api as sm

# Используем <<Заданные параметры>> для определения объема выборки для сплита 50/50 и запуска эксперимента.
# Эксперимент рассчитан для идеальных условий.
# Можно "Поиграться" в <<Заданных параметрах>> и менять значения

# "О чём эксперимент?" 
# Мы моделируем эксперимент в организации. 
# У нас есть объявления, приходит продакт и говорит: 
# "Есть новая фича, необходимо проанализировать его влияние на CTR, прогнозируем увеличение CTR на 2%"
# Мы говорим "Окей" и моделируем эксперимент:
#     1) Определяем объем выборки, который сможет задетектировать MDE по "Заданным параметрам";
#     2) Определяем, какой у нас объем трафика;
#     3) Делим трафик на две группы;
#     4) В первой группе, у нас базовый CTR;
#     5) Во второй группе определяем возможность увеличения CTR на 2%;
#     6) Расчитываем CTR;
#     7) Проводим тест;
# Для надежности эксперимента мы перемешиваем рандомно основной входящий трафик, 
# после чего делим его для тестовой и контрольной группы.


#------------------------------------------------------------------
# <<Заданные параметры>>
base_ctr = 0.03   # базовый CTR
MDE = 0.02        # минимальный эффект который мы хотим обнаружить
alpha = 0.05      # уровень значимости
n = 0.8           # мощность теста
m = 5000          # количество пользователей в день
t = m/2           # делим трафик по группам

#------------------------------------------------------------------
# Расчет стандартного отклонения для Z-теста на равенство долей
pooled_prob = (base_ctr + base_ctr + MDE) / 2
std_dev = math.sqrt(2 * pooled_prob * (1 - pooled_prob))

# Расчет Z-значения для уровня значимости alpha
Z_alpha = norm.ppf(1 - alpha/2)

# Расчет объема выборки для каждой группы
n = (2 * (std_dev * Z_alpha + std_dev * norm.ppf(n))**2) / MDE**2
n = math.ceil(n)

print("Объем выборки для каждой группы в сплит-тесте:", n)

#------------------------------------------------------------------

# Генерация идентификаторов пользователей
user_ids = ['{:011d}'.format(i) for i in range(1, n*2+1)]

# Генерация дат для пользователей, зависит от трафика и объема группы
dates = []
for i in range(n*2):
    dates.append(pd.Timestamp('2022-01-01') + pd.DateOffset(days=i // m))

# Создание датафрейма df
df = pd.DataFrame({'userid': user_ids, 'show': 0, 'click': 0, 'dt': dates})

# Создание пустых датафреймов df0 и df1
df0 = pd.DataFrame(columns=['userid', 'show', 'click', 'dt'])
df1 = pd.DataFrame(columns=['userid', 'show', 'click', 'dt'])

    
for date in df['dt'].unique():
    
    #перемешиваем данные рандомно за дни последовательно в цикле
    users_day = df[df['dt'] == date]['userid'].unique()
    np.random.shuffle(users_day)
    
    #делим перемешанные данные по группам 
    sampls = len(users_day) // 2
    users0 = users_day[:sampls]
    users1 = users_day[sampls:]
    
    # Отбор строк из фрейма данных по значениям в группы 
    users_df0 = df[df['userid'].isin(users0)]
    users_df1 = df[df['userid'].isin(users1)]
    
    # добавление данных за день в группы
    df0 = pd.concat([df0, users_df0], ignore_index=True)
    df1 = pd.concat([df1, users_df1], ignore_index=True)

# Генерируем маркеры показа и клика для контрольной и тестовой групп
# В контрольной группе оставляем базовый мараметр CTR, так как он обычно нам известен
# В тестовую группу добавляем эффект который мы хотим обнаружить
df0['show'] = np.random.choice([0, 1], size=len(df0), p=[1-t/m, t/m])
df0['click'] = np.where((df0['show'] == 1) & (np.random.rand(len(df0)) < base_ctr), 1, 0)

df1['show'] = np.random.choice([0, 1], size=len(df1), p=[1-t/m, t/m])
df1['click'] = np.where((df1['show'] == 1) & (np.random.rand(len(df1)) < base_ctr + MDE), 1, 0)

#------------------------------------------------------------------

# Рассчитываем CTR для контрольной и тестовой групп
control_clicks = df0['click'].sum()
control_shows = df0['show'].sum()
control_ctr = control_clicks / control_shows

test_clicks = df1['click'].sum()
test_shows = df1['show'].sum()
test_ctr = test_clicks / test_shows

# Проводим статистический тест (например, z-тест)
z_score, p_value = sm.stats.proportions_ztest([control_clicks, test_clicks], [control_shows, test_shows])

# Вывод результатов
print(f"CTR в контрольной группе: {control_ctr}")
print(f"CTR в тестовой группе: {test_ctr}")
print(f"Z-статистика: {z_score}")
print(f"P-значение: {p_value}")

if p_value < 0.05:
    print("Различие в CTR статистически значимо")
else:
    print("Различие в CTR не является статистически значимым")

Объем выборки для каждой группы в сплит-тесте: 3014
CTR в контрольной группе: 0.02339572192513369
CTR в тестовой группе: 0.046984572230014024
Z-статистика: -3.472544338409275
P-значение: 0.0005155498455473644
Различие в CTR статистически значимо


C:\Users\Дима\AppData\Local\Temp\ipykernel_8584\718430631.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df0 = pd.concat([df0, users_df0], ignore_index=True)
C:\Users\Дима\AppData\Local\Temp\ipykernel_8584\718430631.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df1 = pd.concat([df1, users_df1], ignore_index=True)


Есть три товара по 10, 50 и 100 рублей. Было сделано 1000 покупок. Вы собрали описательные статистики:

* Средний чек 9,8
* Дисперсия 34.8
* Мода 10
* Самая большая покупка 70р
Все ли с ними хорошо, поясните почему

Есть две гипотезы, которые сразу можно определить, когда мы видим эти значения:
- Гипотеза первая: товары "цельные", и тогда в полученных показателях закралась ошибка при подсчете этих данных. Почему мы так считаем и какие могут быть ошибки?

Почему мы так считаем? Разберем каждый показатель по отдельности:

1. Мода - 10, вопросов нет: есть товар дешевый за 10р, покупок больше, все логично!
2. Самая большая покупка - 70р, что тоже возможно, при чеке: 10+10+50 и тд в разных вариациях.
3. Средний чек - 9.8 такой показатель возможен, если у нас чек без товаров,но это маловероятно! - первый вопросик к данным!
4. Дисперсия, 34.8 она возможна, если примем к сведению, то что средний чек нереалистичен в условиях данной гипотезы и его опустим.
5. Есть товар в 100р, если принять к сведению то, что максимальный чек = 70р, то получается, что его вообще не покупали. Тогда вопрос, зачем выставляли этот товар! - второй вопросик к данным!

Какие могут быть ошибки?:
1. Возможна ошибка при загрузке источников - подтянулись не те данные и "подтянулись" данные из других источников!
2. Возможно при процессе предобработки данных скрипт выполнил не те агрегации и данные сьехали!
3. Возможно при джоине справочников(по метаданным) произошел сбой и подтянулись пустые чеки!
4. Возможно при выгрузке аналитиком, он подгрузил их не из той таблице!

Вариантов много, но вывод один: данные не согласуются с гипотезой и , если действительно товары 'цельные', необходимо более детально изучить данные и выяснить, что могло повлиять на такие результаты!

Рассмотрим вторую гипотезу: данные "Весовые", то есть у нас есть товары за 10, 50, 100р - где ценна указанна за определенную долю веса, например кг. Рассмотрим в данном случае описательные статистики!:
1. Мода -10, вопросов нет: есть товар дешевый за 10р, покупок больше, все логично!
2. Самая большая покупка - 70р, такой показатель возможен набором различных товаров!
3. Средний чек 9.8, такой показатель возможен, скорее всего тавары не частого применения(например специи) и нужды в больших количествах нет! например взяли все по 5 гр
4. Дисперсия 34.8 возможна, стандартное отклонение: корень из 34.8 = 5.9 - скорее всего больше 90% всех наблюдений(общий чек) лежит: 0<90% наблюдений< 19.5 (5.9*1.64+9.8)
Вывод по второй гипотезе: Товары с ценами 10, 50, 100 можно различным образом комбинировать в одном чеке, при этом если они весовые. Данные реальны при второй гипотезе.

Таким образом, так как у нас нет точки отсчета, в каком виде у нас представленны товары, по которым собиралась статистика, я больше всего склоняюсь к тому что у нас верна гипотеза под номером два - товары весовые, такая описательная статистика возможна! 
Если, при анализе оказалось, что товары цельные, то данные нереальны: возможно ошибки при обработке данных, дубли или ошибки иного характера при получении данных из источника.

In [347]:
import numpy as np
import statistics 
# Создаем список чисел для расчета дисперсии
data = [1, 10, 10,  70, 2, 4.6, 3, 6.8, 5, 1, 10, 10, 0.1]


# Рассчитываем статистики
m = np.mean(data)
var = np.var(data)
std = np.std(data)
mad = statistics.mode(data)


In [349]:
mad

10

In [350]:
m

10.26923076923077

In [351]:
var

310.49751479289944

In [ ]:
Есть три товара по 10, 50 и 100 рублей. Было сделано 1000 покупок. Вы собрали описательные статистики:

* Средний чек 9,8
* Дисперсия 34.8
* Мода 10
* Самая большая покупка 70р

In [334]:
import numpy as np

# Генерируем случайные данные
np.random.seed(42)  # Задаем seed для воспроизводимости результатов
observations = np.random.normal(0, 1, 1000)  # Генерируем данные с нулевым средним и единичной дисперсией

# Корректируем данные, чтобы среднее значение было равно 9.8 и дисперсия равна 34.8
observations = observations - np.mean(observations) + 9.8
observations = observations * np.sqrt(34.8 / np.var(observations))

# Рассчитываем статистики
mean_observation = np.mean(observations)
variance_observation = np.var(observations)
std_deviation_observation = np.std(observations)

print("Среднее значение:", mean_observation)
print("Дисперсия:", variance_observation)
print("Стандартное отклонение:", std_deviation_observation)



Среднее значение: 59.068301084702995
Дисперсия: 34.8
Стандартное отклонение: 5.89915248150105
